# PoliDuckies
A notebook with the complete pipeline

### Standard imports

In [37]:
import os 
branch = "daffy" #@param ['master', 'daffy']

if not os.path.isdir('gym-duckietown') and not os.path.isdir('../gym-duckietown'):
  !git clone --branch {branch} https://github.com/duckietown/gym-duckietown.git
  !pip3 install -e gym-duckietown
if "/gym-duckietown" not in os.getcwd():
  os.chdir('gym-duckietown')
  
!pip install numpy==1.20
!apt install xvfb -y
!pip3 install pyvirtualdisplay
!pip install pyglet==1.5.14
!pip install duckietown-gym-daffy
!pip install poliduckie-segmentation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp39-cp39-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.14 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.7.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.20.0 which is incompatible.
ml-dtypes 0.0.4 requires numpy>1.20, but you have numpy 1.20.0 which is incompatible.
librosa 0.10.0.post2 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.20.0 which is incompatible.
jaxlib 0.4.7+cuda11.cudnn86 requires numpy>=1.21, but you have numpy 1.20.0 which is incompati

In [38]:
from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
%matplotlib inline

## used to copy the model to the gdrive folder
import shutil

import cv2
import gym
import gym_duckietown
import pyglet

# Gym has a built in logger to log information. The information is split in importance levels, from highest to lowest:
# CRITICAL, ERROR, WARNING, INFO, DEBUG, NOTSET
# To stop gym from printing many verbose NOTSET, INFO and DEBUG messages we let it print only level WARNING or higher messages
import logging
gym_duckietown.logger.setLevel(logging.WARNING)

In [39]:
map_name = "Duckietown-small_loop-v0"

### Methods

In [40]:
from poliduckie_segmentation.segmentation import Segmentation

seg = Segmentation()

def segmentation(image):
  """
    Returns the image from the simulation with the segmentation

    Image: np.Array
    ------
    Image: np.Array
  """
  resized = cv2.resize(image, (320, 240))
  return seg.predict(resized)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 240, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 240, 320, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 240, 320, 16  448         ['lambda[0][0]']                 
                                )                                                                 
                                                                                              

In [41]:
!wget https://github.com/poliduckie/poliduckie_pipeline/blob/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl?raw=true
!mv SegmentationOutput.pkl?raw=true SegmentationOutput.pkl

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


--2023-04-04 18:04:13--  https://github.com/poliduckie/poliduckie_pipeline/blob/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl?raw=true
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/poliduckie/poliduckie_pipeline/raw/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl [following]
--2023-04-04 18:04:13--  https://github.com/poliduckie/poliduckie_pipeline/raw/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/poliduckie/poliduckie_pipeline/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl [following]
--2023-04-04 18:04:14--  https://raw.githubusercontent.com/poliduckie/poliduckie_pipelin

In [42]:
from poliduckie_segmentation import BirdEyeView

!wget https://github.com/poliduckie/poliduckie_pipeline/blob/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl?raw=true
matrixPath = './SegmentationOutput.pkl'
birdeye = BirdEyeView(path = matrixPath)

def birdEyeTransform(image):
  """
    Returns the image from the simulation with the bird eye view

    Image: np.Array
    ------
    Image: np.Array
  """
  return birdeye.computeBirdEye(image)

--2023-04-04 18:04:14--  https://github.com/poliduckie/poliduckie_pipeline/blob/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl?raw=true
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/poliduckie/poliduckie_pipeline/raw/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl [following]
--2023-04-04 18:04:15--  https://github.com/poliduckie/poliduckie_pipeline/raw/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/poliduckie/poliduckie_pipeline/SingleFixedDistanceReference/notebooks/BirdEyeMatrices/SegmentationOutput.pkl [following]
--2023-04-04 18:04:15--  https://raw.githubusercontent.com/poliduckie/poliduckie_pipelin

In [43]:
from poliduckie_segmentation.line_extraction import LineExtraction

lineExtraction_ = LineExtraction()

def lineExtraction(image, n_points=100, mode="bezier", bezier_degree=3):
  """
    Returns the central line extracted as a numpy array. The mode can be either "bezier" or "spline"

    Image: np.Array
    n_points: int
    mode: str
    bezier_degree: int
    ------
    Line: np.Array
  """
  if mode == "bezier":
    bezier_points = lineExtraction_.bezier_fit(image, degree=bezier_degree, nPoints=n_points)
    return bezier_points
  
  elif mode == "spline":
    spline_points = lineExtraction_.spline_interpolation(image, nPoints=n_points)
    return spline_points

In [44]:
METER_PER_PIXEL = 0.4 / 150
CAR_POSITION = [320/2, 240]

def computeReference(line, N):
  """
    Returns the references in meters
    Reference is a list with dimension (2, N)
    N is the prediction horizon of the MPC

    Line: np.Array
    N: int
    ------
    reference: List[[List[Float], List[Float]]]
  """
  reference = line[:, :N]

  #center around car position
  reference[0] = [x - CAR_POSITION[0] for x in reference[0]]
  reference[1] = [y - CAR_POSITION[1] for y in reference[1]]

  #convert to meters
  reference[0] = [x * METER_PER_PIXEL for x in reference[0]]
  reference[1] = [y * METER_PER_PIXEL for y in reference[1]]
  
  return reference

In [45]:
from poliduckie_segmentation.control import MPC
N = 10
M = MPC(N=10)

def runMpc(state, reference):
  """
    Returns the control inputs to give to the car.
    State is [x, y, theta, v, w]
    Reference is a list with dimension (2, N)

    state: [Float, Float, Float, Float, Float]
    reference: List[[List[Float], List[Float]]]
    ------
    u: List[Float, Float]
  """
  return M.mpc(state, reference)

In [46]:
imageVisualization = 'Plt' #@param ['Plt', 'CV2']

def plt_imshow(image):
  plt.imshow(image)
  plt.show()


if imageVisualization == 'CV2':
  from google.colab.patches import cv2_imshow
  imshow = cv2_imshow
else:
  imshow = plt_imshow

### Simulation

In [47]:
display = Display(visible=0, size=(1400, 900))
display.start()
window = pyglet.window.Window()
env = gym.make(map_name)
env.unwrapped.user_tile_start = [0,1]
env.unwrapped.start_pose = [[0.21699990272521976, 0, 0.41029359288296474], np.pi/2]
imshow(env.reset())

GLException: ignored

In [ ]:
from poliduckie_segmentation.model import Model
import casadi as ca

VERTICAL_CUTOFF = 150

# State X = [x, y, angle, longitudinal speed, angular speed]
state = ca.DM([0,0,0,0,0])

F = Model()

image_recording = []

for _ in range(100):
  image = env.render(mode="rgb_array")
  image_BGR = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  imshow(image)

  image_recording.append(image)
  image_segmentation = segmentation(image_BGR)[0]*255
  imshow(image_segmentation)

  image_birdeye = birdEyeTransform(image_segmentation)
  imshow(image_birdeye)

  dottedLineMask = cv2.threshold(image_birdeye[:, :, 1], 70, 255, cv2.THRESH_BINARY)[1]
  imshow(dottedLineMask)

  dottedLineMaskCropped = dottedLineMask[VERTICAL_CUTOFF:, :]

  line = lineExtraction(dottedLineMaskCropped, mode='bezier', n_points=N+1)
  plt.imshow(dottedLineMask)
  plt.plot([0, 320], [VERTICAL_CUTOFF, VERTICAL_CUTOFF], color='red', linewidth=3, label='cutoff')
  plt.plot(line[0], line[1] + VERTICAL_CUTOFF, color='blue', linewidth=5, label='extracted line')
  plt.legend()
  plt.show()

  reference = computeReference(line, N+1)
  print(reference)
  
  action = runMpc(state, reference)
  state = F.step(*state.toarray().reshape(-1), action)
  env.step(action)